In [9]:
import os
import sys

import numpy as np
import pandas as pd
import cv2
import glob
import gdcm
import yaml
import torch
import pydicom
import zipfile
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO('C:\\Users\\vscha\\Section3\\yolov8\\runs\\detect\\my_yolov8_2\\weights\\best.pt')

In [28]:
images_path = glob.glob('.\\dataset\\test\\*.png')
images_path[1]

'.\\dataset\\test\\0005d3cc-3c3f-40b9-93c3-46231c3eb813.png'

In [30]:
tmp_image = imread(images_path[1])
tmp_image

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 0, 0]], dtype=uint8)

In [31]:
images_path[1].split('\\')[-1][:-4]

'0005d3cc-3c3f-40b9-93c3-46231c3eb813'

In [96]:
image = cv2.cvtColor(cv2.imread(images_path[735]), cv2.COLOR_BGR2RGB)
result = model(image)
for res in result:
    boxes = res.boxes
    masks = res.masks
    probs = res.probs

boxes


0: 640x640 3 pneumonias, 196.0ms
Speed: 3.0ms preprocess, 196.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[3.9099e+02, 1.4577e+02, 5.4808e+02, 4.9166e+02, 3.6293e-01, 0.0000e+00],
        [1.3882e+02, 1.1545e+02, 3.2310e+02, 4.1703e+02, 3.4502e-01, 0.0000e+00],
        [1.5188e+02, 1.7579e+02, 3.0272e+02, 3.8705e+02, 2.8730e-01, 0.0000e+00]], device='cuda:0')
cls: tensor([0., 0., 0.], device='cuda:0')
conf: tensor([0.3629, 0.3450, 0.2873], device='cuda:0')
data: tensor([[3.9099e+02, 1.4577e+02, 5.4808e+02, 4.9166e+02, 3.6293e-01, 0.0000e+00],
        [1.3882e+02, 1.1545e+02, 3.2310e+02, 4.1703e+02, 3.4502e-01, 0.0000e+00],
        [1.5188e+02, 1.7579e+02, 3.0272e+02, 3.8705e+02, 2.8730e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: tensor([640, 640], device='cuda:0')
shape: torch.Size([3, 6])
xywh: tensor([[469.5369, 318.7180, 157.0841, 345.8870],
        [230.9555, 266.2410, 184.2809, 301.5808],
        [227.3006, 281.4174, 150.8431, 211.2564]], device='cuda:0')
xywhn: tensor([[0.7337, 

In [100]:
len(boxes)

3

In [75]:
x,y,w,h = boxes.xywh[1]

In [94]:
boxes.xywh[0].cpu().numpy()

array([     469.54,      318.72,      157.08,      345.89], dtype=float32)

In [105]:
boxes.conf[0].cpu().numpy()

array(    0.36293, dtype=float32)

In [126]:
def inference(lst, model):
  predicts = []

  for k in tqdm(range(len(lst))):
    image = cv2.cvtColor(cv2.imread(lst[k]), cv2.COLOR_BGR2RGB)
    patient_id = (lst[k].split('\\')[-1][:-4])
    bboxes = ''

    results = model(image)
    for result in results:
      boxes = result.boxes  # Boxes object for bbox outputs
      for i in range(len(boxes)):
        x,y,w,h = boxes.xywh[i].cpu().numpy()
        confidence = boxes.conf[i].cpu().numpy()
        bbox = ' '.join(map(str, [confidence, x, y, w, h])) + ' '
        bboxes += bbox
    bboxes = bboxes.rstrip()

    line = {'patientId':patient_id,
            'PredictionString':bboxes}
    predicts.append(line)

  df_pred = pd.DataFrame(columns=['patientId', 'PredictionString'], data=predicts)
  return df_pred
     

In [136]:
point = []

AttributeError: 'list' object has no attribute 'is_null'

In [141]:
def adjust_ratio(string):
  strings = string.split('\n')
  img_width = 1024
  img_height = 1024

  ls = ''
  for point in strings:
    if point == '':
      continue

    point = point.split()
    point = [float(i) for i in point]

    for i in range(len(point)//4):
      if len(point[i*5:i*5+5])!=5:
        continue
      conf, x_center, y_center, width, height = point[i*5:i*5+5]

      # Convert relative coordinates to pixel-based coordinates
      x_center_pixel = x_center/640 * img_width
      y_center_pixel = y_center/640 * img_height
      width_pixel = width/640 * img_width
      height_pixel = height/640 * img_height

      # Calculate the pixel-based bounding box coordinates
      conf = float(conf)
      x_min_pixel = int(round(x_center_pixel - (width_pixel / 2), 0))
      y_min_pixel = int(round(y_center_pixel - (height_pixel / 2), 0))
      width_pixel = int(round(width_pixel, 0))
      height_pixel = int(round(height_pixel, 0))

      ans = ' '.join(map(str, [conf, x_min_pixel, y_min_pixel, width_pixel, height_pixel])) + ' '
      ls += ans

  ls = ls.rstrip()
  return ls

In [113]:
df_pred = inference(images_path, model)
df_pred.iloc[735]

  0%|          | 0/3000 [00:00<?, ?it/s]


0: 640x640 (no detections), 181.0ms
Speed: 2.0ms preprocess, 181.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 14.0ms
Speed: 0.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.0ms
Speed: 0.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 11.0ms
Speed: 0.0ms preprocess, 11.0ms

patientId                        116fee2c-9f15-401a-8a7f-c260a6f5a8e0
PredictionString    0.36293086 469.53693 318.71802 157.0841 345.88...
Name: 735, dtype: object

In [142]:
df_pred['PredictionString'] = df_pred['PredictionString'].apply(adjust_ratio)

In [145]:
df_pred.to_csv('./submission.csv', index=False)